# Initialisation

In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import numpy as np
import os
from PIL import Image
import lmfit
%matplotlib tk

# Importer fichier

In [2]:
path_to_tiff = os.path.join("..", "acquisition", "video_output_carac_150ms_1im_1um.tiff")

tiff = Image.open(path_to_tiff)

# Nombre de frames pas vide

In [3]:
with Image.open(path_to_tiff) as img:
    frame_number = 0
    actual_frames = 0
    try:
        while True:
            frame_number += 1
            if np.sum(np.array(img)) != 0:
                actual_frames += 1
                
            img.seek(frame_number)
    except EOFError:
        print("All frames processed.")

actual_frames

All frames processed.


71

# 1re frame

**Le nombre de frames ignorés n'est pas pris en compte**

In [4]:
frame_index = 0
tiff.seek(frame_index)
original_image = np.array(tiff)

while np.sum(original_image) == 0:
    frame_index += 1
    tiff.seek(frame_index)
    original_image = np.array(tiff)
    print(frame_index)

1


# Traitement d'image

In [5]:
clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
preprocessed = clahe.apply(original_image)

blurred = cv2.medianBlur(preprocessed, 115)
preprocessed2 = cv2.subtract(preprocessed, blurred)

# Apply Non-Local Means Denoising
img = cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Sélection du point à tracker

In [6]:
def crop(img, x, y, crop_size=100):
    x_start = int(x - crop_size // 2)
    x_end = int(x + crop_size // 2)
    y_start = int(y - crop_size // 2)
    y_end = int(y + crop_size // 2)

    return img[y_start:y_end, x_start:x_end]

In [7]:
# Display the image and let the user select a point interactively
fig, ax = plt.subplots()
ax.imshow(img, cmap='gray')  # Use 'gray' for better visibility of grayscale images
plt.title(f"Frame {frame_index}: Select a point")

# Ask for a point to be selected
print("Please click on the point you want to select.")
x, y = plt.ginput(1)[0]  # This will get the coordinates of the clicked point
print(f"Selected point: ({x}, {y})")
plt.close()

crop_sze = 100

Please click on the point you want to select.
Selected point: (442.5742129008896, 570.0094001702893)


# Fit gaussien sur le point sélectionné

In [8]:
def prepare_data(x, y, z):
    return (x.flatten(), y.flatten()), z.flatten()

In [9]:
def gaussian_2d(xy, amplitude, x0, y0, sigma_x, sigma_y, offset):
    x, y = xy
    a = 1 / (2 * sigma_x**2)
    b = 1 / (2 * sigma_y**2)
    return offset + amplitude * np.exp(- (a * (x - x0)**2 + b * (y - y0)**2))

In [10]:
def localisateur_gaussien(intensity_grid, maxi):
    x = np.arange(intensity_grid.shape[0])
    y = np.arange(intensity_grid.shape[1])
    X, Y = np.meshgrid(x, y)

    # Préparer les données pour le fit
    (xdata, ydata), zdata = prepare_data(X, Y, intensity_grid)
    model = lmfit.Model(gaussian_2d)
    max_idx = np.unravel_index(np.argmax(intensity_grid), intensity_grid.shape)
    initial_x0 = x[max_idx[0]]
    initial_y0 = y[max_idx[1]]

    # Définir les paramètres du modèle
    params = model.make_params(
        amplitude=np.max(intensity_grid),
        x0=initial_x0,
        y0=initial_y0,
        sigma_x=1,
        sigma_y=1,
        offset=2
    )

    # Effectuer l'ajustement
    result = model.fit(zdata, params, xy=(xdata, ydata))

    x_position = result.params['x0'].value + maxi[0] - 24.5
    y_position = result.params['y0'].value + maxi[1] - 24.5

    return [x_position, y_position], result.params['sigma_x'].value, result.params['sigma_y'].value

# Process d'image (enlever le bruit)

**semble faire du trouble**

In [11]:
def denoise(image):
    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
    preprocessed = clahe.apply(image)
    
    blurred = cv2.medianBlur(preprocessed, 115)
    preprocessed2 = cv2.subtract(preprocessed, blurred)
    
    return cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Passe au prochain frame

In [12]:
def next_frame(frame_index):
    frame_index += 1
    tiff.seek(frame_index)
    original_image = np.array(tiff)
    
    while np.sum(original_image) == 0:
        frame_index += 1
        tiff.seek(frame_index)
        original_image = np.array(tiff)

    return [frame_index, original_image]

# Faire le crop et fit

In [13]:
def particle_tracker(image, x, y, frame_index):
    image = denoise(image)
    
    cropped_img = crop(image, x, y, crop_sze)

    result_fit = localisateur_gaussien(cropped_img, [x, y])

    x_new, y_new = result_fit[0][0], result_fit[0][1]

    return [result_fit, cropped_img, (x_new, y_new), (result_fit[1], result_fit[2])]

# Main loop

In [14]:
position_list = []
sigma_list = []
crop_frames = []
big_frames = []

for _ in range(actual_frames - 1):
    data = particle_tracker(img, x, y, frame_index)
    position_list.append(data[2])
    sigma_list.append(data[3])
    frame_index, img = next_frame(frame_index)
    big_frames.append(img)
    crop_frames.append(data[1])

# Résultats

In [15]:
position_list

[(np.float64(468.7651814922473), np.float64(594.5428668702059)),
 (np.float64(471.7276632319729), np.float64(600.6586031303884)),
 (np.float64(475.0458324462252), np.float64(607.7033640740087)),
 (np.float64(503.625608668681), np.float64(588.8695006951373)),
 (np.float64(484.40146645757625), np.float64(600.108583622248)),
 (np.float64(459.5397946532712), np.float64(560.4446483529156)),
 (np.float64(477.94629803231203), np.float64(613.764035490999)),
 (np.float64(505.69422939999913), np.float64(591.1661238576139)),
 (np.float64(506.9661895197163), np.float64(597.5212333339709)),
 (np.float64(506.0946255035899), np.float64(569.0537778997862)),
 (np.float64(500.12711013639773), np.float64(605.4553065447037)),
 (np.float64(-11994.20428957374), np.float64(576.7553657293998)),
 (np.float64(-480.10583410548395), np.float64(575.4508492517572)),
 (np.float64(-15345.226618631512), np.float64(576.0590673534973)),
 (np.float64(-7807.362475788953), np.float64(573.0236759804926)),
 (np.float64(506.9

In [16]:
sigma_list

[(np.float64(3.2048482443533084), np.float64(4.343094504303097)),
 (np.float64(3.0966976973507783), np.float64(4.809020895109953)),
 (np.float64(-2.958671161174716), np.float64(-4.580959823711562)),
 (np.float64(-10.698249204472878), np.float64(95.44858291473166)),
 (np.float64(-1.8428326595930646), np.float64(-0.3112297671596119)),
 (np.float64(59.145265692678414), np.float64(22.646713471369385)),
 (np.float64(2.4965621729124177), np.float64(4.1936539195410125)),
 (np.float64(9.386864434311946), np.float64(45.74804175133058)),
 (np.float64(9.751836231746827), np.float64(47.03548507289736)),
 (np.float64(11.80429264555293), np.float64(77.30549380409302)),
 (np.float64(1.3048885979138758), np.float64(-0.194539028694731)),
 (np.float64(7141.444923986571), np.float64(31.456125448600684)),
 (np.float64(577.4435781215673), np.float64(30.379652418995143)),
 (np.float64(-4208.172128757842), np.float64(-78.28004872249184)),
 (np.float64(-4793.646450236605), np.float64(-35.274304955823496)),
 (

In [17]:
x_plt, y_plt = zip(*position_list)
# Create the plot
plt.figure(figsize=(10, 8))  # Optional: Adjust the figure size
plt.plot(x_plt, y_plt, marker='o', linestyle='-', color='b', label='Connected Points')

# Set grid limits to match the 1440x1080 grid
plt.xlim(0, 1440)
plt.ylim(0, 1080)

# Add labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('2D Connected Points Plot')
plt.legend()

# Optional: Add grid lines
plt.grid(True)

## anim toutes les particules

In [18]:
# fig, ax = plt.subplots()
# img = ax.imshow(big_frames[0], cmap='gray', animated=True)


# # Update function
# def update(frame):
#     img.set_array(frame)
#     return img,
    
# ani = animation.FuncAnimation(fig, update, frames=big_frames, interval=50, blit=True)
# plt.show()

## anim crop

In [19]:
fig, ax = plt.subplots()
img = ax.imshow(crop_frames[0], cmap='gray', animated=True)


def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=crop_frames, interval=50, blit=True)
plt.show()